In [0]:
%pip install Pdfplumber langchain
dbutils.library.restartPython()

In [0]:
from pyspark.sql.functions import substring_index

directory_path = "/Volumes/raginm/rag_llm/ragfiles"

#list all the files in the directory
files_paths = [file.path for file in dbutils.fs.ls(directory_path)]
#print(files_paths)

df = spark.createDataFrame(files_paths, "string").select(substring_index("value", "/", -1).alias("file_name"))

df.display()

In [0]:
import os
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdf_volume_path = "/Volumes/raginm/rag_llm/ragfiles"

# get the list of already processed pdf files

processed_files = spark.sql(f"SELECT DISTINCT file_name FROM raginm.rag_llm.docs_load_tracker").collect()

processed_files = set(row["file_name"] for row in processed_files)

# process new pdf files only

new_files = [file for file in os.listdir(pdf_volume_path) if file not in processed_files]

all_text = '' # initialize all_text to store text from new pdf files

for file_name in new_files:
    pdf_path = os.path.join(pdf_volume_path, file_name)

    with pdfplumber.open(pdf_path) as pdf:
        for pdf_page in pdf.pages:
            single_page_text = pdf_page.extract_text()
            all_text = all_text + '\n' + single_page_text


#split the text into chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

length_function = len

splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=length_function
)

chunks = splitter.split_text(all_text)

print(chunks)




In [0]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import ArrayType, StringType
import pandas as pd

@pandas_udf("array<string>")
def get_chunks(dummy):
    return pd.Series([chunks])

# Register the UDF
spark.udf.register("get_chunks_udf", get_chunks)

In [0]:
%sql
insert into raginm.rag_llm.docs_text  (text)
select explode(get_chunks_udf('dummy')) as text

In [0]:
df.createOrReplaceTempView("temp_table")  # Create a temporary table from the DataFrame

# Insert only the rows that do not exist in the target table
spark.sql("""
    INSERT INTO raginm.rag_llm.docs_load_tracker
    SELECT * , current_timestamp FROM temp_table
    WHERE NOT EXISTS (
        SELECT 1 FROM raginm.rag_llm.docs_load_tracker
        WHERE temp_table.file_name = raginm.rag_llm.docs_load_tracker.file_name
    )
""")